In [649]:
import os
import pandas as pd
import re

In [650]:
mobility_path = '../data/raw/'
processed_path = '../data/processed/'
raw_path = '../data/raw/'


In [651]:
# mobility_combine = []

# for file in os.listdir(mobility_path):
#     df = pd.read_csv(path + file, sep='\t')
#     df = df.loc[(df['Kind'] == 'subregion')]
#     mobility_combine.append(df)

# mobility_df_combine = pd.concat(mobility_combine)

# mobility_df_combine.to_csv(path+'mobility_county.csv')

In [652]:
mobility = pd.read_csv(mobility_path+'Global_Mobility_Report.csv', low_memory=False)
processed = pd.read_csv(processed_path + 'time_series.csv')
zips = pd.read_csv(raw_path + 'uszips.csv')

In [625]:
# clean USA data
USA = mobility.loc[mobility['country_region'] == 'United States'].reset_index()
to_drop = ['index', 'country_region_code', 'country_region']
USA = USA.drop(to_drop, axis=1)
USA.insert(1, 'county', USA['sub_region_2'].str.lower())
USA = USA.drop(['sub_region_2'],axis=1)
USA = USA.loc[~pd.isna(USA['county'])]
USA['county'] = USA['county'].map(lambda x: re.sub(' county| city','', x)).str.lower()
USA.insert(1, 'state', USA['sub_region_1'].str.lower())
USA = USA.drop(['sub_region_1'],axis=1)

In [626]:
processed['county'] = processed['county'].str.lower()
processed['state'] = processed['state'].str.lower()

In [627]:
# take only subset of counties according to proccessed data
counties = pd.unique(processed['county'].str.lower().map(lambda x: re.sub(' city','', x)))
USA = USA.loc[USA['county'].str.lower().isin(counties)]

In [628]:
# difference in counties
difference = set(pd.unique(counties)) - set(USA['county'].str.lower().unique())

In [629]:
# Cut time
USA = USA.loc[USA['date']<'2020-04-05']

In [632]:
# deal with duplicates
columns = set(USA.columns) - set(['state', 'county', 'date'])
agg = {key:'mean' for key in columns}
USA_grouped = USA.groupby(['state', 'county', 'date']).aggregate(agg)

In [643]:
df = pd.merge(USA_grouped,processed, on=['state', 'county','date'])

In [644]:
len(df[['date', 'county', 'county_fips_all', 'timestamp']].drop_duplicates())-len(df)

0

In [645]:
df.to_csv(processed_path + 'time_series_mobility.csv')